# Identify Single Cell Anomalies
Here we train Anomalyze models on different pre-computed datasets.

### Import Libraries

In [ ]:
import pathlib
import sys

import joblib
import pandas as pd
import pyarrow.parquet as pq
from sklearn.ensemble import IsolationForest

## Define paths

### Inputs

In [1]:
plate_data_name = pathlib.Path(sys.argv[1]).name
sampled_plate_jump_data_path = sys.argv[2]

sampled_platedf = pd.read_parquet(
    f"{sampled_plate_jump_data_path}/{plate_data_name}.parquet"
)

--------------------------------------------------------------
FileNotFoundError            Traceback (most recent call last)
Cell In[7], line 4
      1 big_drive_path = f"{root_dir}/big_drive"
      3 # Plate morphology data
----> 4 plate_paths = pathlib.Path(sys.argv[1]).resolve(strict=True)
      6 # Boolean flag for if the data is single-cell
      7 is_sc = sys.argv[2].lower() == "true"

File ~/mambaforge-pypy3/envs/jump_sc/lib/python3.9/pathlib.py:1215, in Path.resolve(self, strict)
   1209 def resolve(self, strict=False):
   1210     """
   1211     Make the path absolute, resolving all symlinks on the way and also
   1212     normalizing it (for example turning slashes into backslashes under
   1213     Windows).
   1214     """
-> 1215     s = self._flavour.resolve(self, strict=strict)
   1216     if s is None:
   1217         # No symlink resolution => for consistency, raise an error if
   1218         # the path doesn't exist or is forbidden
   1219         self.stat()

File

### Outputs

In [ ]:
isoforest_path = pathlib.Path("isolation_forest_models")
isoforest_path.mkdir(parents=True, exist_ok=True)

isoforest_path = pathlib.Path(
    isoforest_path / f"{plate_data_name}_isolation_forest.joblib"
)

## Train Anomalyze Models

In [ ]:
meta_cols = [col for col in sampled_platedf.columns if "Metadata" in col]
featdf = sampled_platedf.drop(columns=meta_cols).dropna(axis=1, how="any")

# If 1_600 trees are trained with 256 samples per tree, then
# 1_600 * 256 gives approximately the expected number of samples per tree.
# For some of the plate data, this number of samples can barely fit in memory.
# We also want to maximize the number of trees to learn many patterns for identifying anomalies.
# 256 is empirically the largest number of samples per tree that allowed outliers to be isolated better.
isofor = IsolationForest(n_estimators=1_600, random_state=0, n_jobs=-1)
isofor.fit(featdf)

joblib.dump(isofor, isoforest_path)